# Project: Stochastic and Spatial Models

## 0. Init

In [ ]:
# Arrays and analysis
import numpy as np
import scipy as sp
from scipy.integrate import solve_ivp
from scipy.optimize import curve_fit
import pandas as pd
import bisect

# Plotting and config
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = [6, 3]
plt.rcParams['lines.linewidth'] = 1
plt.rcParams['figure.constrained_layout.use'] = True

# Misc imports
from slugify import slugify
from functools import partial
import os

# Important directories
FIG_DIR = 'fig/'
DATA_DIR = 'data/'

def save_fig(title):
    """Save figure under normalized name."""
    plt.savefig(f'{FIG_DIR}/{slugify(title)}.png', bbox_inches='tight')

def update_ax(props_dict):
    """Update current Axes with the given dictionary of properties."""
    plt.gca().update(props_dict)

def create_dirs(path):
    """Create directory, do nothing if it exists."""
    os.makedirs(path, exist_ok=True)

In [ ]:
def initialize():
    """Initialize project."""
    create_dirs(FIG_DIR)
    create_dirs(DATA_DIR)

initialize()

In [ ]:
rng = np.random.default_rng()

In [ ]:
def sir_first_reaction(y0, params, t_max):
    """Implementation of Gillespie's Direct Algorithm (DA)."""
    X, Y, Z = y0
    beta, gamma, mu = params
    
    # 1.
    # Events and results.
    ps = [
        ["birth", (1, 0, 0)],
        ["transmission", (-1, 1, 0)],
        ["recovery", (0, -1, 1)],
        ["death_X", (-1, 0, 0)],
        ["death_Y", (0, -1, 0)],
        ["death_Z", (0, 0, -1)],
    ]

    # Bookkeeping.
    t = 0
    ts = []
    ys = []
    while t < t_max:    
        # 2.
        N = np.sum([X, Y, Z])
        Rs = [mu*N, beta*X*Y/N, gamma*Y, mu*X, mu*Y, mu*Z]
        
        # 3.
        R_total = np.sum(Rs)
        
        # 4.
        rand_1 = rng.random()
        dt = -1 / R_total * np.log(rand_1)
        
        # 5.
        rand_2 = rng.random()
        P = rand_2 * R_total
        
        # 6.
        R_cum = np.cumsum(Rs)
        
        # Find index of event p.
        p_idx = np.searchsorted(R_cum, P)
        
        # Extract result from p.
        p = ps[p_idx]
        result = np.array(p[1])

        X, Y, Z = np.add([X, Y, Z], result)
        
        # 7.
        t += dt

        # Bookkeep results.
        ts.append(t); ys.append([X, Y, Z])

    return np.array(ts), np.array(ys)

In [ ]:
ts, ys = sir_first_reaction(y0=(990, 10, 0), params=(3, 1, 1/80), t_max=100)

In [ ]:
# np.array(ys).shape

fig, ax = plt.subplots()

plt.plot(ts, ys[:, 0], color="r", label="X")
plt.plot(ts, ys[:, 1], color="g", label="Y")
plt.plot(ts, ys[:, 2], color="b", label="Z")

update_ax({"xlabel": "t (days)", "ylabel": "population", "title": "DA"})
plt.legend()
plt.show()